In [1]:
#The MIT License (MIT)

#Copyright (c) 2020 Juliana T.C. Marcos

#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO
#THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE 
#AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF 
#CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

#This code use methods of the ParticleFilter (PF) class in order to track an object in a video. The PF uses two
#measurements providers which are intermitently chosen based on the value of the ssim index of two images. The
#first image is a template of the animal to track while the second is the Region of Interest (ROI) surrounding
#the previous estimated location of the tracked animal. The measurements providers are a colour image segmentation
#technique and the You Only Look Once (YOLO) object detector. 

#The particles are represented in red, the PF estimate of
#the tracked animal is represented in green and the animal's bb is represented in blue. 

#Thanks to Nayak for the nice tutorial about using YOLOv3 with OpenCv which is available at this address:
#https://www.learnopencv.com/deep-learning-based-object-detection-using-yolov3-with-opencv-python-c/


In [2]:
#Import of useful librairies
import cv2
import math
import numpy as np
from skimage import measure
from ParticleFilter import ParticleFilter
import time

In [3]:
"""Some variables initialization """

#The total number of trials
Tot=1
#Lists for the Tot running outputs averages 
BB_avg=[]
ROI_avg=[]
anchor_avg=[]
xy_est_avg=[]
ssim_avg=[]
particles_avg=[]
#number of particles
n_particles=2000
#noise in sensors' measurements
meas_noise=0
#Read weights and config files to create YOLO(v3) net
net = cv2.dnn.readNet("./Inputs/yolov3.weights", "./Inputs/yolov3.cfg")
#Fetch the three output layers names of YOLO net, they are the ones not connected to 
#any following layers since they are the last layers
layer_names = net.getLayerNames()
output_layer_names = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
#Read the classes in coco.names file for YOLO net
coco_classes = []
with open ('./Inputs/coco.names','r') as file:
    coco_classes=[line.strip("\n") for line in file.readlines()]

font = cv2.FONT_HERSHEY_SIMPLEX
text_coord=(10,40)
t_size=0.8
t_thick=2
#This value was chosen according to a paper experiment
N_thresh=(2*n_particles)/30
#Scale for YOLO's inputs preprocessing
scale=1/255
#Video frame width and height
frame_width=1920
frame_height=1080
#Lists to contain the counters for each trial
cis_l=[]
yolo_l=[]
yolo_det_l=[]
n_resampl_l=[]
#Initialize variable for shifting the anchor update between first measurements and first estimations
anchor_shift=60

In [4]:
#Data for video cows
anchorS=(1250,350)
#This threshold helps to filter small contours
#It is however important to adapt it to the object scales in the videos
Area_thresh=0
#This is to choose the type of threshold (between cv2.THRESH_BINARY_INV 
#and THRESH_BINARY)
type_thr=cv2.THRESH_BINARY_INV
videoIn_name="./Inputs/cows.avi"
path="./Outputs/"
videoOut_name=path+"cows-pf-ssim-colour-yolo.avi"
template = cv2.imread('./Inputs/template2.png')
template2 = cv2.imread('./Inputs/template2_grass.png')
#std in the prediction of particles for the object's position
std=10
chg_thres=0.60
#These are YOLO parameters
conf=0.8
nms=0.7
height, width, channels =224,320,3
blob_x,blob_y=224,320
handover=path+"handpics/cows"
#Motion model's speed in x and y directions
v_x=0.01
v_y=0.01

In [5]:
handover

'./Outputs/handpics/cows'

In [6]:
fyi,fxi=template2.shape[0],template2.shape[1]

In [7]:
start_time = time.time()
for num in range(Tot):
    #List for averaging running outputs
    BB_l=[]
    ROI_l=[]
    anchor_l=[]
    xy_est_l=[]
    ssim_l=[]
    particles_l=[]
    #Counters for call of cis, yolo and yolo actual detection
    cis=0
    yolo=0
    yolo_det=0
    it=0
    n_resampling=0
    #A single program variables initialization
    anchor=anchorS
    #Initialization of measurements variables
    x_objMeasure,y_objMeasure=anchor[0],anchor[1]
    #Initialization of some important variables
    ssim=0
    #BB variable initialization
    BB=0,0,0,0
    #Capture video where object tracking should be performed
    video = cv2.VideoCapture(videoIn_name)
    #Video output
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_output = cv2.VideoWriter(videoOut_name,fourcc,60,(frame_width,frame_height))
    #Particles Instantiation
    particles=ParticleFilter(frame_width,frame_height, n_particles)

    #This function uses opencv functions to identify the center of the animal to track
    def sensors_measurements(template,img,anchor,meas_noise=3,kernel=3):

        #Compute the mean color of the template containing the animal to track color
        meanStdTemplate=cv2.meanStdDev(template)

        #Convert the color of the frame to work on and apply the GaussianBlur function in order to remove
        #Gaussian noise, smooth image and somrtimes highlight edges
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.GaussianBlur(img, (kernel, kernel), 0)

        #Binarize frame using the color mean of the animal in such a way that matching parts
        #of the frame will appear white and other parts will appear black
        #Use the green channel to find the threshold value since it works well empirically
        ret,thresh = cv2.threshold(img,meanStdTemplate[0][1],255,type_thr)

        # find contours in the thresholded image. The if condition is used to avoid error that happens depending
        #on the python version used. The middle parameter is to only retrieve parent contours while the latter
        #is to avoid redundant points in contours.
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = cnts[0] if len(cnts) == 2 else cnts[1]

        #Compute the contours areas
        contoursAreas=[cv2.contourArea(c) for c in contours]

        #Compute the center and the BB of the contours
        contoursCenter=[]
        contoursBB=[]

        for c in range(len(contours)):
            M = cv2.moments(contours[c])
            #If the area of the current contour exists and is greater than a threshold for areas 
            if M["m00"] != 0 and contoursAreas[c]>Area_thresh:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                contoursCenter.append((c,cX,cY,contoursAreas[c]))
                contoursBB.append(cv2.boundingRect(contours[c]))

        #Compute the distance between each center and the anchor center in order to find 
        #the most suitable shape to track i.e the closest one to the anchor

        contoursAnchorDist=[math.sqrt((contoursCenter[i][1]-anchor[0])**2+\
        (contoursCenter[i][2]-anchor[1])**2) for i in range (len(contoursCenter))]

        #Save and return the coordinates of the most suitable center and its contours
        index=contoursAnchorDist.index(min(contoursAnchorDist))
        cX=contoursCenter[index][1]+np.random.standard_normal()*meas_noise
        cY=contoursCenter[index][2]+np.random.standard_normal()*meas_noise

        return cX,cY,contoursBB[index]
    
    #Loop through the entire video
    while (True):
        #Take the video and break it frame by frame
        _,frame=video.read()
        #Check if frames are captured
        if(_ == False ): break

        it+=1
        #Particles prediction update
        particles.particles_update(v_x,v_y,std,frame_width,frame_height)
        particles_l.append(particles.particles.copy())
        anchor_l.append(anchor)
    
        #Take the ROI around the animal in the current frame for the SSIM evaluation 
        #with the template and its dimensions
        ROI_ssim=(int(anchor[0]-fxi/2),int(anchor[1]-fyi/2),fxi,fyi)
        frame_crop=frame[ROI_ssim[1]:ROI_ssim[1]+ROI_ssim[3],ROI_ssim[0]:ROI_ssim[0]+ROI_ssim[2]]
        fy,fx=frame_crop.shape[0],frame_crop.shape[1]
    
        #Here, we check if the dimensions of the ROI around the animal and the template are the same
        #to select the cis if not, because these cases correspond to the frame boundaries. Therefore, 
        #the ROI might not contain sufficient information to allow the change detection with the ssim.
        if ((fxi!=fx) or (fyi!=fy)):
            ssim=1.1
        else: 
            #The value 3 is the minimum dimension for images to compute the ssim index
            if ((fxi>3 and fyi>3)): 
                ssim=measure.compare_ssim(frame_crop,template2, \
                                              multichannel=True,win_size=3)
            else:
                ssim=0.0
        
        ssim=abs(ssim)
        ssim_l.append(ssim)
        
        if (ssim<chg_thres):
            
            yolo+=1

            #Coordinates of ROI around anchor
            dy1=int(anchor[1]-height/2)
            #if(dy1<0):dy1=0
            dy2=int(anchor[1]+height/2)
            #if (dy2>frame_height):dy2=frame_height
            dx1=int(anchor[0]-width/2)
            #if (dx1<0): dx1=0
            dx2=int(anchor[0]+width/2)
            #if (dx2>frame_width): dx2=frame_width

            frame_roi=frame[dy1:dy2,dx1:dx2]
            ROI_l.append((dx1,dy1,dx2,dy2))
            
            # Detecting objects
            blob = cv2.dnn.blobFromImage(frame_roi,scale,(blob_x, blob_y),(0, 0, 0), True, crop=False)
            net.setInput(blob)
            #Run forward pass to get predictions from output layers selected
            outputs = net.forward(output_layer_names)
            # Showing informations on the screen
            classes = []
            confidences = []
            boxes = []
            yolo_center=[]
            yolo_center_anchor_dist=[]

            #for each output of the YOLO last layers
            for output in outputs:
                for detection in output:
                #Each detection contains center point x,y,width,height,object probability
                #and 80 class probability
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > conf:
                        print("step ",str(it),"confidence ",confidence,"\n")
                        #non normalized detected object's center, width and height 
                        center_x,center_y = int(detection[0] *blob_x),int(detection[1] * blob_y)
                        w,h = int(detection[2] * blob_x),int(detection[3] * blob_y)
                        # Rectangle coordinates
                        x,y = int(center_x - w / 2),int(center_y - h / 2)
                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        classes.append(class_id)

                        #Get coordinates in initial video frame reference instead of 
                        #ROI reference
                        center_x_reverse=center_x+(anchor[0]-width/2)
                        center_y_reverse=center_y+(anchor[1]-height/2)
                        yolo_center.append((center_x_reverse,center_y_reverse))

            #Search and keep relevant bounding boxes given their scores 
            indexes = cv2.dnn.NMSBoxes(boxes, confidences, conf, nms)
            #Only compute the distances between anchor and the yolo predictions
            #if confidence is good and indexes is not empty
            if (len(indexes)!=0):
                yolo_center_anchor_dist=[math.sqrt((yolo_center[i][0]-anchor[0])**2+\
                (yolo_center[i][1]-anchor[1])**2) for i in range (len(yolo_center))\
                                        if i in indexes]
                #Keep the relevant indices in a list to retrieve the relevant index latter
                nms_indexes=[i for i in range (len(yolo_center)) if i in indexes]

                print("step ",str(it),"indexes ",indexes," nms ",nms_indexes,\
                " yolo_center ",yolo_center," distances ", yolo_center_anchor_dist, "\n")

                #Save and return the coordinates of the most suitable center and its BB
                #Keep the index of the minimum distance between prediction and anchor
                index=yolo_center_anchor_dist.index(min(yolo_center_anchor_dist))
                #Retrieve relevant index to select the yolo index and the corresponding BB
                index_yolo=nms_indexes[index]
                print("step ",str(it)," index ",index," index_yolo ",index_yolo,"\n")
                BB_x=int(boxes[index_yolo][0]+(anchor[0]-width/2))
                BB_y=int(boxes[index_yolo][1]+(anchor[1]-height/2))
                BB =BB_x,BB_y,boxes[index_yolo][2],boxes[index_yolo][3]

                #Update measurements  
                x_objMeasure,y_objMeasure=yolo_center[index_yolo][0],yolo_center[index_yolo][1]
                
                yolo_det+=1
            
            cv2.rectangle(frame, (dx1,dy1), (dx2,dy2), (0,0,255), 2)
            cv2.putText(frame,'Change detected call yolo: SSIM = '+str(ssim),text_coord,font,t_size,(255,255,255),t_thick,cv2.LINE_AA)   

        else :
            #Measurements
            x_objMeasure,y_objMeasure,BB = sensors_measurements(template,frame,anchor,meas_noise)
            #Use anchor to select the closest area and then the actual tracked target
            cis+=1
            ROI_l.append((0,0,0,0))
            cv2.putText(frame,'No Change detected, use CIS: SSIM = '+str(ssim),text_coord,font,t_size,(255,255,255),t_thick,cv2.LINE_AA)   

        #Save BB in a list
        BB_l.append(BB)
        
        #Update the particles weights with the new measurement
        particles.weigth_update(x_objMeasure,y_objMeasure)

        #Estimation of object center position
        x_estimation,y_estimation=particles.position_estimation()

        if it%200==0:
            np.save(handover+"_estimation_data2_"+str(it)+".npy",np.array([x_estimation,y_estimation]))
            np.save(handover+"_particles_data2_"+str(it)+".npy",np.array(particles.particles))
            np.save(handover+'_weights_data2_'+str(it)+'.npy',np.array(particles.weights))
            cv2.imwrite(handover+'_HandoverA_data2_'+str(it)+'.png',frame)
        

        #Save the x and y estimated in a list
        xy_est_l.append((x_estimation,y_estimation))

        #Draw the particles
        particles.draw_box_particles(frame,BB,x_estimation,y_estimation)
        #Draw the position estimation
        cv2.circle(frame,(x_estimation,y_estimation),5,[0,255,0],3)
       
        #Update the anchor with either the current measurements or the x and y estimates
        if it > anchor_shift:
            anchor=x_estimation,y_estimation
        else:
            anchor=x_objMeasure,y_objMeasure
        
        #Resample the particles
        if (particles.effective_particles() < N_thresh):
            n_resampling+=1
            particles.resampling()
            
        #Reinitialize BB to avoid cis bb drawing when yolo does not detect
        BB=0,0,0,0

        if (num==Tot-1):
            video_output.write(frame)

        #if it==100:
        #    break
    
    cis_l.append(cis)
    yolo_l.append(yolo)
    yolo_det_l.append(yolo_det) 
    n_resampl_l.append(n_resampling)
    BB_avg.append(BB_l)
    xy_est_avg.append(xy_est_l)
    ssim_avg.append(ssim_l)
    particles_avg.append(particles_l)
    ROI_avg.append(ROI_l)
    anchor_avg.append(anchor_l)
    

step  534 confidence  0.946042 

step  534 indexes  [[0]]  nms  [0]  yolo_center  [(1288.0, 436.0)]  distances  [82.73451516749222] 

step  534  index  0  index_yolo  0 

step  536 confidence  0.88970876 

step  536 indexes  [[0]]  nms  [0]  yolo_center  [(1297.0, 441.0)]  distances  [90.80198235721508] 

step  536  index  0  index_yolo  0 

step  537 confidence  0.856421 

step  537 indexes  [[0]]  nms  [0]  yolo_center  [(1293.0, 439.0)]  distances  [72.44998274671983] 

step  537  index  0  index_yolo  0 

step  540 confidence  0.9134652 

step  540 indexes  [[0]]  nms  [0]  yolo_center  [(1294.0, 438.0)]  distances  [71.47027354082255] 

step  540  index  0  index_yolo  0 

step  541 confidence  0.9380592 

step  541 confidence  0.9174439 

step  541 indexes  [[0]
 [1]]  nms  [0, 1]  yolo_center  [(1290.0, 428.0), (1415.0, 394.0)]  distances  [14.866068747318506, 117.27744881263405] 

step  541  index  0  index_yolo  0 

step  542 confidence  0.8286925 

step  542 confidence  0.891

step  614 confidence  0.88459533 

step  614 confidence  0.8295161 

step  614 indexes  [[0]
 [1]]  nms  [0, 1]  yolo_center  [(1455.0, 376.0), (1335.0, 449.0)]  distances  [141.4814475470194, 8.94427190999916] 

step  614  index  1  index_yolo  1 

step  615 confidence  0.91053426 

step  615 indexes  [[0]]  nms  [0]  yolo_center  [(1341.0, 459.0)]  distances  [7.810249675906654] 

step  615  index  0  index_yolo  0 

step  616 confidence  0.905483 

step  616 indexes  [[0]]  nms  [0]  yolo_center  [(1344.0, 460.0)]  distances  [9.219544457292887] 

step  616  index  0  index_yolo  0 

step  617 confidence  0.8086139 

step  617 confidence  0.93179995 

step  617 indexes  [[1]
 [0]]  nms  [0, 1]  yolo_center  [(1456.0, 376.0), (1346.0, 450.0)]  distances  [140.91131963046828, 8.48528137423857] 

step  617  index  1  index_yolo  1 

step  618 confidence  0.97255456 

step  618 indexes  [[0]]  nms  [0]  yolo_center  [(1353.0, 462.0)]  distances  [12.041594578792296] 

step  618  index  

step  685 confidence  0.8509756 

step  685 confidence  0.86784524 

step  685 indexes  [[1]
 [0]]  nms  [0, 1]  yolo_center  [(1358.0, 443.0), (1511.0, 463.0)]  distances  [147.31259280862582, 10.0] 

step  685  index  1  index_yolo  1 

step  686 confidence  0.9096714 

step  686 confidence  0.96390384 

step  686 indexes  [[1]
 [0]]  nms  [0, 1]  yolo_center  [(1359.0, 439.0), (1503.0, 470.0)]  distances  [151.08275877809487, 7.810249675906654] 

step  686  index  1  index_yolo  1 

step  687 confidence  0.94437927 

step  687 confidence  0.94029504 

step  687 indexes  [[0]
 [1]]  nms  [0, 1]  yolo_center  [(1358.0, 442.0), (1510.0, 472.0)]  distances  [150.0966355385756, 6.4031242374328485] 

step  687  index  1  index_yolo  1 

step  688 confidence  0.9301636 

step  688 confidence  0.92087007 

step  688 indexes  [[0]
 [1]]  nms  [0, 1]  yolo_center  [(1359.0, 446.0), (1512.0, 475.0)]  distances  [150.0966355385756, 6.4031242374328485] 

step  688  index  1  index_yolo  1 

step

step  723 confidence  0.8806869 

step  723 indexes  [[0]]  nms  [0]  yolo_center  [(1501.0, 470.0)]  distances  [10.63014581273465] 

step  723  index  0  index_yolo  0 

step  724 confidence  0.8791748 

step  724 confidence  0.83616126 

step  724 indexes  [[0]
 [1]]  nms  [0, 1]  yolo_center  [(1363.0, 451.0), (1512.0, 478.0)]  distances  [143.02796929272262, 8.54400374531753] 

step  724  index  1  index_yolo  1 

step  725 confidence  0.87777436 

step  725 indexes  [[0]]  nms  [0]  yolo_center  [(1362.0, 450.0)]  distances  [145.16886718577092] 

step  725  index  0  index_yolo  0 

step  726 confidence  0.89001757 

step  726 indexes  [[0]]  nms  [0]  yolo_center  [(1520.0, 469.0)]  distances  [13.45362404707371] 

step  726  index  0  index_yolo  0 

step  727 confidence  0.86880606 

step  727 confidence  0.937718 

step  727 indexes  [[1]
 [0]]  nms  [0, 1]  yolo_center  [(1368.0, 450.0), (1521.0, 467.0)]  distances  [146.972786596703, 10.63014581273465] 

step  727  index  

step  810 confidence  0.83140093 

step  810 indexes  [[0]]  nms  [0]  yolo_center  [(1574.0, 474.0)]  distances  [13.45362404707371] 

step  810  index  0  index_yolo  0 

step  811 confidence  0.91611373 

step  811 indexes  [[0]]  nms  [0]  yolo_center  [(1574.0, 485.0)]  distances  [8.602325267042627] 

step  811  index  0  index_yolo  0 

step  812 confidence  0.89773864 

step  812 indexes  [[0]]  nms  [0]  yolo_center  [(1577.0, 488.0)]  distances  [8.48528137423857] 

step  812  index  0  index_yolo  0 

step  813 confidence  0.89892644 

step  813 indexes  [[0]]  nms  [0]  yolo_center  [(1585.0, 473.0)]  distances  [15.620499351813308] 

step  813  index  0  index_yolo  0 

step  814 confidence  0.8168583 

step  814 indexes  [[0]]  nms  [0]  yolo_center  [(1585.0, 487.0)]  distances  [8.48528137423857] 

step  814  index  0  index_yolo  0 

step  815 confidence  0.8069394 

step  815 indexes  [[0]]  nms  [0]  yolo_center  [(1595.0, 468.0)]  distances  [17.69180601295413] 

st

In [8]:
prog_duration= time.time() - start_time
prog_duration

268.30003118515015

In [9]:
prog_duration/60

4.471667186419169

In [10]:
prog_duration/(60*Tot)

4.471667186419169

In [11]:
sum(n_resampl_l)/Tot

420.0

In [12]:
len(BB_l),len(xy_est_l),len(BB_avg),len(xy_est_avg),len(particles_avg),len(ssim_avg),\
len(anchor_avg),len(ROI_avg)

(980, 980, 1, 1, 1, 1, 1, 1)

In [13]:
sum(cis_l)/Tot,sum(yolo_l)/Tot,sum(yolo_det_l)/Tot

(699.0, 281.0, 192.0)

In [14]:
BB_l_avg=[] 
BB_avg=np.array(BB_avg)
BB_l_avg=np.sum(BB_avg,0)/Tot

In [15]:
xy_est_l_avg=[] 
xy_est_avg=np.array(xy_est_avg)
xy_est_l_avg=np.sum(xy_est_avg,0)/Tot
xy_est_l_avg

array([[1125.,  472.],
       [1226.,  394.],
       [1244.,  355.],
       ...,
       [1371.,  427.],
       [1371.,  428.],
       [1371.,  427.]])

In [16]:
ssim_l_avg=[] 
ssim_avg=np.array(ssim_avg)
ssim_l_avg=np.sum(ssim_avg,0)/Tot

In [17]:
ROI_l_avg=[] 
ROI_avg=np.array(ROI_avg)
ROI_l_avg=np.sum(ROI_avg,0)/Tot

In [18]:
particles_l_avg=[] 
particles_avg=np.array(particles_avg)
particles_l_avg=np.sum(particles_avg,0)/Tot

In [19]:
anchor_l_avg=[] 
anchor_avg=np.array(anchor_avg)
anchor_l_avg=np.sum(anchor_avg,0)/Tot
anchor_l_avg

array([[1250.,  350.],
       [1250.,  364.],
       [1252.,  364.],
       ...,
       [1372.,  426.],
       [1371.,  427.],
       [1371.,  428.]])

In [20]:
#xy_est_l

In [21]:
#BB_l

In [22]:
np.save(path+'xy_data2.npy',np.array(xy_est_l_avg))
np.save(path+'BB_data2.npy',np.array(BB_l_avg))
np.save(path+'ssim_data2.npy',np.array(ssim_l_avg))
np.save(path+'ROI_data2.npy',np.array(ROI_l_avg))
np.save(path+'part_data2.npy',np.array(particles_l_avg))
np.save(path+'anchor_data2.npy',np.array(anchor_l_avg))